In [ ]:
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()
cwd

In [ ]:
def NRZ_maker(values):
    NRZ = []
    state = None
    prior = values[0]

    for bit in values:
        if bit == prior:
            state = 1.0
        else:
            state = 0.0
        NRZ.append(state)
    return NRZ

In [ ]:
test_array = np.array([6, 6, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12])
test_df = pd.DataFrame({'values': test_array})
values = test_df['values'].tolist()

NRZ = []
state = 0
prior = values[0]
for bit in values:
    if bit == prior:
        state = 1.0
    else:
        state = 0.0
    NRZ.append(state)
#NRZ

test_df['NRZ'] = NRZ
test_df.head()

In [ ]:
values = test_df['values'].tolist()
test_df['NRZ2'] = NRZ_maker(values)
test_df.head()

In [ ]:
PATH = '/data22/bde'
netflix_StrangerThings = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/Season3-StrangerThings/'
netflix_animated_RickandMorty = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/animated-RickandMorty/'
amazon_TheExpanse = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/Season3-TheExpanse/'
amazon_animated_AdventureTime = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/animated-AdventureTime/'
download = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Download/Driving/'
df_list = []
file_dir = download

for file in os.listdir(file_dir)[:]:
    print(file)
    df = pd.read_csv(file_dir+file)

    values = df['CellID'].tolist()
    df['NRZ'] = NRZ_maker(values)

    #df['CellIDChange'] = (df['CellID'] != df['CellID'].shift()).astype('float32')

    df = df[['Timestamp', 'CellID', 'NRZ', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate']]
    df = df.replace('-', np.nan)
    
    df['date'] = pd.to_datetime(df['Timestamp'], format="%Y.%m.%d_%H.%M.%S")
    df = df.drop(columns=['Timestamp'])

    df['CellID'] = pd.to_numeric(df['CellID'], downcast='float')
    df['RSRP'] = pd.to_numeric(df['RSRP'], downcast='float')
    df['RSRQ'] = pd.to_numeric(df['RSRQ'], downcast='float')
    df['SNR'] = pd.to_numeric(df['SNR'], downcast='float')
    df['CQI'] = pd.to_numeric(df['CQI'], downcast='float')
    df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='float')
    df['DL_bitrate'] = pd.to_numeric(df['DL_bitrate'], downcast='float')
    df['UL_bitrate'] = pd.to_numeric(df['UL_bitrate'], downcast='float')

    df.fillna(method='ffill', inplace=True)
    df.fillna('0', inplace=True)

    df.drop_duplicates(subset=['date'], inplace=True)
    df.info()
    #df.info(show_counts=True)
    print(df.head(30))
    #df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
    #df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)
    df_list.append(df)

In [ ]:
df = pd.concat(df_list, axis=0)
df.sort_values('date', inplace=True)
df.info()

In [ ]:
df.head()

In [ ]:
df = df[['date', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'CellID', 'CellIDChange', 'UL_bitrate', 'DL_bitrate']]

In [ ]:
df.to_csv('data/5Gdataset/download.csv', encoding='utf-8', index=False)